## Mattis' code

### 1. Imports

In [1]:
# Base imports
import pandas as pd
import numpy as np
import os
import sys
import cv2

# Add the parent directory to sys.path
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Tensorflow imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import optimizers

# Scikit-learn imports
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Custom imports
from classification.ml_logic.preprocessor import Preprocessor

In [2]:
preprocessor = Preprocessor()

### 2. Data imports

In [4]:
# Define file path
MASTER_PATH = '../raw_data/MINI-DDSM-Complete-JPEG-8'

# Load Excel file
df = pd.read_excel(os.path.join(MASTER_PATH, 'DataWMask.xlsx'))
print(f'Data loaded successfully. {len(df)} records found.')
print(df.head())

# Replace backslashes with forward slashes in fullPath
df['fullPath'] = df['fullPath'].str.replace('\\', '/', regex=False)
# Ensure fullPath is a string
df['fullPath'] = df['fullPath'].astype(str)

# Filter out mask files (keep only original images)
df_images = df[~df['fileName'].str.contains('Mask', na=False)]
print(f'Filtered images: {len(df_images)} records remaining after removing masks.')

# Extract patiend ID from fileName (format: C_{patient_id}_1_LATERALITY_VIEW.jpg)
df_images['fileName'] = df_images['fileName'].astype(str).str.strip()
df_images['patient_id'] = df_images['fileName'].str.extract(r'\w_(\d+)_1')

# Create full image paths
#df_images['full_image_path'] = df_images['fullPath']

# Binary mapping: Cancer = 1, Benign and Normal = 0
def create_binary_labels(status):
    if status == 'Cancer':
        return 1
    else: # Benign or Normal
        return 0

df_images['binary_label'] = df_images['Status'].apply(create_binary_labels)
print(df_images.tail())

# Remove any rows with missing labels
df_images = df_images.dropna(subset=['binary_label', 'patient_id'])
print(f'Final dataset size after removing missing labels: {len(df_images)} records.')

print(f"Total images: {len(df_images)}")
print(f"Total unique patients: {df_images['patient_id'].nunique()}")
print(f"Original class distribution:\n{df_images['Status'].value_counts()}")
print(f"\nBinary class distribution:")
print(f"Non-Cancer (Benign + Normal): {len(df_images[df_images['binary_label'] == 0])}")
print(f"Cancer: {len(df_images[df_images['binary_label'] == 1])}")

Data loaded successfully. 7808 records found.
                             fullPath                fileName View   Side  \
0    Benign\0029\C_0029_1.LEFT_CC.jpg    C_0029_1.LEFT_CC.jpg   CC   LEFT   
1   Benign\0029\C_0029_1.LEFT_MLO.jpg   C_0029_1.LEFT_MLO.jpg  MLO   LEFT   
2   Benign\0029\C_0029_1.RIGHT_CC.jpg   C_0029_1.RIGHT_CC.jpg   CC  RIGHT   
3  Benign\0029\C_0029_1.RIGHT_MLO.jpg  C_0029_1.RIGHT_MLO.jpg  MLO  RIGHT   
4    Benign\0033\C_0033_1.LEFT_CC.jpg    C_0033_1.LEFT_CC.jpg   CC   LEFT   

   Status                          Tumour_Contour Tumour_Contour2   Age  \
0  Benign   Benign\0029\C_0029_1.LEFT_CC_Mask.jpg               -  66.0   
1  Benign  Benign\0029\C_0029_1.LEFT_MLO_Mask.jpg               -  66.0   
2  Benign                                       -               -  66.0   
3  Benign                                       -               -  66.0   
4  Benign                                       -               -  60.0   

   Density  
0        3  
1        3  
2

### 3. Patient-based Train/Val/Test split

In [5]:
def patient_based_split(df_images, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15, random_state=42):
    """
    Split data by patient ID to avoid data leakage.
    Args:
        df_images (pd.DataFrame): DataFrame containing image data with 'patient_id' and 'Status'.
        train_ratio (float): Proportion of data to use for training.
        val_ratio (float): Proportion of data to use for validation.
        test_ratio (float): Proportion of data to use for testing.
        random_state (int): Random seed for reproducibility.
    Returns:
        tuple: DataFrames for train, validation, and test sets.
    """
    # Get unique patients with their status
    patient_info = df_images.groupby('patient_id')['Status'].first().reset_index()
    patient_info['binary_label'] = patient_info['Status'].apply(create_binary_labels)

    unique_patients = patient_info['patient_id'].unique()

    # Shuffle patients
    np.random.seed(random_state)
    np.random.shuffle(unique_patients)

    # Calculate split sizes
    total_patients = len(unique_patients)
    train_size = int(train_ratio * total_patients)
    val_size = int(val_ratio * total_patients)

    # Split patient IDs
    train_patients = unique_patients[:train_size]
    val_patients = unique_patients[train_size:train_size+val_size]
    test_patients = unique_patients[train_size+val_size:]

    print(f"\nPatient distribution:")
    print(f"Train patients: {len(train_patients)}")
    print(f"Validation patients: {len(val_patients)}")
    print(f"Test patients: {len(test_patients)}")

    # Assign split labels to all images based on patient ID
    def assign_split(patient_id):
        if patient_id in train_patients:
            return 'train'
        elif patient_id in val_patients:
            return 'val'
        else:
            return 'test'

    df_images['split'] = df_images['patient_id'].apply(assign_split)

    # Create separate dataframes
    train_df = df_images[df_images['split'] == 'train'].copy()
    val_df = df_images[df_images['split'] == 'val'].copy()
    test_df = df_images[df_images['split'] == 'test'].copy()

    # Print detailed statistics
    print(f"\nImage distribution:")
    print(f"Train images: {len(train_df)} (Cancer: {sum(train_df['binary_label'])}, Non-Cancer: {len(train_df) - sum(train_df['binary_label'])})")
    print(f"Val images: {len(val_df)} (Cancer: {sum(val_df['binary_label'])}, Non-Cancer: {len(val_df) - sum(val_df['binary_label'])})")
    print(f"Test images: {len(test_df)} (Cancer: {sum(test_df['binary_label'])}, Non-Cancer: {len(test_df) - sum(test_df['binary_label'])})")

    # Verify no patient leakage
    train_patients_set = set(train_df['patient_id'].unique())
    val_patients_set = set(val_df['patient_id'].unique())
    test_patients_set = set(test_df['patient_id'].unique())

    assert len(train_patients_set.intersection(val_patients_set)) == 0, "Patient leakage between train and val!"
    assert len(train_patients_set.intersection(test_patients_set)) == 0, "Patient leakage between train and test!"
    assert len(val_patients_set.intersection(test_patients_set)) == 0, "Patient leakage between val and test!"

    print("✓ No patient leakage detected!")

    return train_df, val_df, test_df

# Execute the split
train_df, val_df, test_df = patient_based_split(df_images)


Patient distribution:
Train patients: 1184
Validation patients: 253
Test patients: 255

Image distribution:
Train images: 5480 (Cancer: 1880, Non-Cancer: 3600)
Val images: 1144 (Cancer: 412, Non-Cancer: 732)
Test images: 1184 (Cancer: 424, Non-Cancer: 760)
✓ No patient leakage detected!


### 4. Data loading function

In [6]:
def load_and_preprocess_data(df, preprocess_image_func):
    """
    Load images and apply preprocessing for a specific split
    Args:
        df (pd.DataFrame): DataFrame containing image paths and labels.
        preprocess_image_func (function): Function to preprocess images.
    Returns:
        tuple: Numpy arrays of images, labels, and patient IDs.
    """
    images = []
    labels = []
    patient_ids = []

    for idx, row in df.iterrows():
        try:
            # Load image
            img_path = os.path.join(MASTER_PATH, row['fullPath'])
            if os.path.exists(img_path):
                image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

                # Apply preprocessing
                processed_image = preprocess_image_func(image)

                images.append(processed_image)
                labels.append(row['binary_label'])
                patient_ids.append(row['patient_id'])
            else:
                print(f"Warning: Image not found at {img_path}")

        except Exception as e:
            print(f"Error processing {row['fileName']}: {str(e)}")

    return np.array(images), np.array(labels), patient_ids


In [7]:
from concurrent.futures import ThreadPoolExecutor
def load_data_concurrently(df, preprocess_image_func, max_workers=4):
    """
    Load images concurrently using ThreadPoolExecutor.
    Args:
        df (pd.DataFrame): DataFrame containing image paths and labels.
        preprocess_image_func (function): Function to preprocess images.
        max_workers (int): Number of threads to use.
    Returns:
        tuple: Numpy arrays of images, labels, and patient IDs.
    """
    images = []
    labels = []
    patient_ids = []

    def process_row(row):
        img_path = os.path.join(MASTER_PATH, row['fullPath'])
        if os.path.exists(img_path):
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            processed_image = preprocess_image_func(image)
            return processed_image, row['binary_label'], row['patient_id']
        else:
            print(f"Warning: Image not found at {img_path}")
            return None

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [executor.submit(process_row, row) for _, row in df.iterrows()]
        for future in futures:
            result = future.result()
            if result is not None:
                img, label, patient_id = result
                images.append(img)
                labels.append(label)
                patient_ids.append(patient_id)

    return np.array(images), np.array(labels), patient_ids

### 5. Create Model

In [8]:
def create_binary_model(input_shape):
    """
    Create CNN model for binary classification (Cancer vs Non-Cancer)
    """
    model = Sequential()

    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(3, 3))

    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(3, 3))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(3, 3))

    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))  # Single output for binary classification

    return model

### 6. Main Training Pipeline

In [9]:
def train_mammography_classifier(train_df, val_df, test_df, preprocess_image_func):
    """
    Complete binary classification training pipeline with patient-based splits
    Args:
        train_df (pd.DataFrame): DataFrame for training set.
        val_df (pd.DataFrame): DataFrame for validation set.
        test_df (pd.DataFrame): DataFrame for test set.
        preprocess_image_func (function): Function to preprocess images.
    Returns:
        tuple: Trained model, training history, and test data.
    """
    # Load and preprocess data for each split
    print("Loading and preprocessing images...")
    X_train, y_train, train_patient_ids = load_data_concurrently(train_df, preprocess_image_func)
    X_val, y_val, val_patient_ids = load_data_concurrently(val_df, preprocess_image_func)
    X_test, y_test, test_patient_ids = load_data_concurrently(test_df, preprocess_image_func)

    print(f"Training samples: {len(X_train)} from {len(set(train_patient_ids))} patients.")
    print(f"Validation samples: {len(X_val)} from {len(set(val_patient_ids))} patients.")
    print(f"Test samples: {len(X_test)} from {len(set(test_patient_ids))} patients.")

    # Create model
    input_shape = X_train.shape[1:]
    model = create_binary_model(input_shape)

    # Compile model
    optimizer = optimizers.Adam(learning_rate=0.001)
    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy', 'recall', 'precision']
    )

    print("\nModel Summary:")
    model.summary()

    # Data augmentation
    datagen = ImageDataGenerator(
        brightness_range=[0.9, 1.1],  # ±10% intensity
        # contrast_range=[0.9, 1.2],    # Enhanced contrast
        channel_shift_range=5.0,      # Color balance variation
        preprocessing_function=lambda x: x + np.random.normal(0, 0.03, x.shape)  # Add Gaussian noise
    )

    # Callbacks
    es = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    plateau = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=0.00001
    )

    # Train model
    print("Starting training...")
    history = model.fit(
        # datagen.flow(X_train, y_train, batch_size=64),
        X_train, y_train, batch_size=64,
        epochs=30,
        # steps_per_epoch=len(X_train) // 32,
        callbacks=[es, plateau],
        validation_data=(X_val, y_val),
        verbose=1
    )

    return model, history, X_train, y_train, X_val, y_val, X_test, y_test

### 7. Model Evaluation function

In [10]:
def evaluate_final_model(model, X_test, y_test):
    """
    Evaluate the trained model on the test set
    Args:
        model (tf.keras.Model): Trained model.
        X_test (np.ndarray): Test images.
        y_test (np.ndarray): Test labels.
    Returns:
        None
    """
    # Make predictions
    y_pred_prob = model.predict(X_test)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()

    # Print classification report
    print("\nFinal Test Set Evaluation:")
    print("="*50)
    target_names = ['Non-Cancer (Benign+Normal)', 'Cancer']
    print(classification_report(y_test, y_pred, target_names=target_names))

    # Confusion matrix
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    print("[[True Non-Cancer, False Cancer]")
    print(" [False Non-Cancer, True Cancer]]")

    # Calculate medical metrics
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # Recall for cancer detection
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # Recall for non-cancer detection
    ppv = tp / (tp + fp) if (tp + fp) > 0 else 0  # Positive Predictive Value
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0  # Negative Predictive Value

    print(f"\nMedical Performance Metrics:")
    print(f"Sensitivity (Cancer Detection Rate): {sensitivity:.3f}")
    print(f"Specificity (Non-Cancer Detection Rate): {specificity:.3f}")
    print(f"Positive Predictive Value (PPV): {ppv:.3f}")
    print(f"Negative Predictive Value (NPV): {npv:.3f}")


### 8. Execute pipeline and evaluate on test set

In [11]:
import tensorflow as tf
import sys

print("Python version:", sys.version)
print("TensorFlow version:", tf.__version__)
print("Physical devices:", tf.config.list_physical_devices())
print("GPU devices:", tf.config.list_physical_devices('GPU'))

# Check if tensorflow-metal is installed
try:
    import tensorflow_metal
    print("tensorflow-metal is installed")
except ImportError:
    print("tensorflow-metal is NOT installed")


Python version: 3.10.6 (main, Jun  2 2025, 11:35:41) [Clang 17.0.0 (clang-1700.0.13.5)]
TensorFlow version: 2.16.2
Physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
tensorflow-metal is NOT installed


In [12]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [13]:
# with tf.device('/physical_device:GPU:O'):
# Execute the pipeline
model, history, X_train, y_train, X_val, y_val, X_test, y_test = train_mammography_classifier(train_df, val_df, test_df, Preprocessor.preprocess_image)

# Evaluate the final model
evaluate_final_model(model, X_test, y_test)

Loading and preprocessing images...
Training samples: 5480 from 1184 patients.
Validation samples: 1144 from 253 patients.
Test samples: 1184 from 255 patients.


/Users/mattisweber/.pyenv/versions/3.10.6/envs/lewagon-breast-cancer-project/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-06-13 15:19:25.859136: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-06-13 15:19:25.861586: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-06-13 15:19:25.862135: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-06-13 15:19:25.862392: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-06-13 15:19:25.862449: I tensorflow/core/common_ru


Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 72, 72, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 22, 22, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 22, 22, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

 Trainable params: 224,641 (877.50 KB)

 Non-trainable params: 224 (896.00 B)

Starting training...
Epoch 1/30


2025-06-13 15:19:27.865978: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


86/86 ━━━━━━━━━━━━━━━━━━━━ 24s 255ms/step - accuracy: 0.5496 - loss: 3.2949 - precision: 0.3709 - recall: 0.4093 - val_accuracy: 0.6294 - val_loss: 2.5418 - val_precision: 0.3571 - val_recall: 0.0364 - learning_rate: 0.0010
Epoch 2/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 248ms/step - accuracy: 0.5844 - loss: 2.5335 - precision: 0.3916 - recall: 0.4185 - val_accuracy: 0.6285 - val_loss: 1.4336 - val_precision: 0.4552 - val_recall: 0.1602 - learning_rate: 0.0010
Epoch 3/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 21s 247ms/step - accuracy: 0.5727 - loss: 2.5434 - precision: 0.3706 - recall: 0.3663 - val_accuracy: 0.6451 - val_loss: 1.1827 - val_precision: 0.5158 - val_recall: 0.2379 - learning_rate: 0.0010
Epoch 4/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 22s 252ms/step - accuracy: 0.5973 - loss: 2.3000 - precision: 0.4027 - recall: 0.3955 - val_accuracy: 0.6469 - val_loss: 1.1844 - val_precision: 0.5465 - val_recall: 0.1141 - learning_rate: 0.0010
Epoch 5/30
86/86 ━━━━━━━━━━━━━━━━━━━━ 22s 256ms/step - accuracy: 0.6110

## v02

In [20]:
def train_mammography_classifier_v2(X_train, y_train, X_val, y_val):
    """
    Complete binary classification training pipeline with patient-based splits
    Args:
        X_train (np.ndarray): Training images.
        y_train (np.ndarray): Training labels.
        X_val (np.ndarray): Validation images.
        y_val (np.ndarray): Validation labels.
    Returns:
        tuple: Trained model, training history.
    """
    # Load data for each split
    print("Loading images...")

    # Create model
    input_shape = X_train.shape[1:]
    model = create_binary_model(input_shape)

    # Compile model
    optimizer = optimizers.Adam(learning_rate=0.001)
    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy', 'recall', 'precision']
    )

    print("\nModel Summary:")
    model.summary()

    # Data augmentation
    datagen = ImageDataGenerator(
        brightness_range=[0.9, 1.1],  # ±10% intensity
        # contrast_range=[0.9, 1.2],    # Enhanced contrast
        channel_shift_range=5.0,      # Color balance variation
        preprocessing_function=lambda x: x + np.random.normal(0, 0.03, x.shape)  # Add Gaussian noise
    )

    # Callbacks
    es = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    plateau = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=0.00001
    )

    # Train model
    print("Starting training...")
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=32),
        # X_train, y_train, batch_size=64,
        epochs=100,
        # steps_per_epoch=len(X_train) // 32,
        callbacks=[es, plateau],
        validation_data=(X_val, y_val),
        verbose=1
    )

    return model, history

### 1. Training with data augmentation

In [21]:
model_v2, history_v2 = train_mammography_classifier_v2(X_train, y_train, X_val, y_val)

# Evaluate the final model
evaluate_final_model(model_v2, X_test, y_test)

Loading images...

Model Summary:


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 222, 222, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 222, 222, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 74, 74, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 72, 72, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 72, 72, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 24, 24, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 22, 22, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 22, 22, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 224,865 (878.38 KB)

 Trainable params: 224,641 (877.50 KB)

 Non-trainable params: 224 (896.00 B)

Starting training...


/Users/mattisweber/.pyenv/versions/3.10.6/envs/lewagon-breast-cancer-project/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 32s 171ms/step - accuracy: 0.5751 - loss: 3.5707 - precision: 0.3873 - recall: 0.3727 - val_accuracy: 0.6407 - val_loss: 1.4776 - val_precision: 0.5027 - val_recall: 0.2233 - learning_rate: 0.0010
Epoch 2/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 29s 166ms/step - accuracy: 0.5737 - loss: 3.2118 - precision: 0.3773 - recall: 0.3639 - val_accuracy: 0.6399 - val_loss: 1.4847 - val_precision: 0.5000 - val_recall: 0.0801 - learning_rate: 0.0010
Epoch 3/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 33s 194ms/step - accuracy: 0.5926 - loss: 2.3901 - precision: 0.3944 - recall: 0.3902 - val_accuracy: 0.6460 - val_loss: 0.9080 - val_precision: 0.5166 - val_recall: 0.2646 - learning_rate: 0.0010
Epoch 4/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 37s 217ms/step - accuracy: 0.5967 - loss: 1.7310 - precision: 0.4176 - recall: 0.4000 - val_accuracy: 0.6469 - val_loss: 0.8772 - val_precision: 0.5400 - val_recall: 0.1311 - learning_rate: 0.0010
Epoch 5/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 37s 214

## v03

### 1. Revised model architecture

In [23]:
from tensorflow.keras.layers import Input, Activation, Add, GlobalAveragePooling2D, Reshape, Multiply
from tensorflow.keras.models import Model


def create_improved_binary_model(input_shape):
    """
    Improved CNN model for mammography binary classification
    """
    inputs = Input(shape=input_shape)

    # Initial feature extraction block
    x = Conv2D(32, (3, 3), activation='relu', padding='same',
               kernel_initializer='he_normal')(inputs)
    x = BatchNormalization()(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.25)(x)

    # Residual block 1
    shortcut1 = Conv2D(64, (1, 1), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Add()([x, shortcut1])
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.25)(x)

    # Residual block 2
    shortcut2 = Conv2D(128, (1, 1), padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same',
               kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Add()([x, shortcut2])
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)

    # Attention mechanism
    attention = GlobalAveragePooling2D()(x)
    attention = Dense(128 // 8, activation='relu')(attention)
    attention = Dense(128, activation='sigmoid')(attention)
    attention = Reshape((1, 1, 128))(attention)
    x = Multiply()([x, attention])

    # Global pooling and final layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.4)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

def train_mammography_classifier_v3(X_train, y_train, X_val, y_val):
    """
    Complete binary classification training pipeline with patient-based splits
    Args:
        X_train (np.ndarray): Training images.
        y_train (np.ndarray): Training labels.
        X_val (np.ndarray): Validation images.
        y_val (np.ndarray): Validation labels.
    Returns:
        tuple: Trained model, training history.
    """
    # Load data for each split
    print("Loading images...")

    # Create model
    input_shape = X_train.shape[1:]
    model = create_improved_binary_model(input_shape)

    # Compile model
    optimizer = optimizers.Adam(learning_rate=0.001)
    model.compile(
        loss='binary_crossentropy',
        optimizer=optimizer,
        metrics=['accuracy', 'recall', 'precision']
    )

    print("\nModel Summary:")
    model.summary()

    # Data augmentation
    datagen = ImageDataGenerator(
        brightness_range=[0.9, 1.1],  # ±10% intensity
        # contrast_range=[0.9, 1.2],    # Enhanced contrast
        channel_shift_range=5.0,      # Color balance variation
        preprocessing_function=lambda x: x + np.random.normal(0, 0.03, x.shape)  # Add Gaussian noise
    )

    # Callbacks
    es = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    )

    plateau = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=0.00001
    )

    # Train model
    print("Starting training...")
    history = model.fit(
        datagen.flow(X_train, y_train, batch_size=32),
        # X_train, y_train, batch_size=64,
        epochs=100,
        # steps_per_epoch=len(X_train) // 32,
        callbacks=[es], # , plateau],
        validation_data=(X_val, y_val),
        verbose=1
    )

    return model, history

In [24]:
model_v3, history_v3 = train_mammography_classifier_v3(X_train, y_train, X_val, y_val)

# Evaluate the final model
evaluate_final_model(model_v3, X_test, y_test)

Loading images...

Model Summary:


Model: "functional_52"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 224, 224,  │        896 │ input_layer_4[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        128 │ conv2d_12[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 224, 224,  │      9,248 │ batch_normalizat… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 224, 224,  │        128 │ conv2d_13[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_12    │ (None, 112, 112,  │          0 │ batch_normalizat… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_4 (Dropout) │ (None, 112, 112,  │          0 │ max_pooling2d_12… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 112, 112,  │     18,496 │ dropout_4[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_15[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 112, 112,  │     36,928 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 112, 112,  │        256 │ conv2d_16[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 112, 112,  │      2,112 │ dropout_4[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 112, 112,  │          0 │ batch_normalizat… │
│                     │ 64)               │            │ conv2d_14[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 112, 112,  │          0 │ add[0][0]         │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_13    │ (None, 56, 56,    │          0 │ activation[0][0]  │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 56, 56,    │          0 │ max_pooling2d_13… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 56, 56,    │     73,856 │ dropout_5[0][0] 

 Total params: 370,545 (1.41 MB)

 Trainable params: 369,137 (1.41 MB)

...

Starting training...


/Users/mattisweber/.pyenv/versions/3.10.6/envs/lewagon-breast-cancer-project/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
  8/172 ━━━━━━━━━━━━━━━━━━━━ 14:30 5s/step - accuracy: 0.4648 - loss: 1.6325 - precision: 0.3717 - recall: 0.4355

/Users/mattisweber/.pyenv/versions/3.10.6/envs/lewagon-breast-cancer-project/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


172/172 ━━━━━━━━━━━━━━━━━━━━ 76s 402ms/step - accuracy: 0.4951 - loss: 1.4432 - precision: 0.3781 - recall: 0.5101 - val_accuracy: 0.6399 - val_loss: 2.9872 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 215s 1s/step - accuracy: 0.5528 - loss: 1.2585 - precision: 0.3731 - recall: 0.4429 - val_accuracy: 0.6399 - val_loss: 0.8923 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.5942 - loss: 1.0558 - precision: 0.4088 - recall: 0.3896 - val_accuracy: 0.6521 - val_loss: 0.8847 - val_precision: 0.5814 - val_recall: 0.1214
Epoch 4/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.6050 - loss: 0.9104 - precision: 0.4031 - recall: 0.3208 - val_accuracy: 0.6434 - val_loss: 0.6932 - val_precision: 0.7500 - val_recall: 0.0146
Epoch 5/100
172/172 ━━━━━━━━━━━━━━━━━━━━ 1577s 9s/step - accuracy: 0.6404 - loss: 0.7263 - precision: 0.4524 - recall: 0.3463 - val_accuracy: 0.63

## TransferLearning

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np

# 1. Enhanced Model Architecture with Focal Loss
def create_transfer_learning_model(input_shape):
    """Create EfficientNet-B0 model with enhanced regularization"""
    base_model = EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )

    # Freeze 75% of initial layers, fine-tune the rest
    for layer in base_model.layers[:-int(len(base_model.layers)*0.25)]:
        layer.trainable = False

    inputs = Input(shape=input_shape)
    x = base_model(inputs)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)  # Increased dropout
    x = Dense(256, activation='relu', kernel_regularizer='l2')(x)  # L2 regularization
    x = Dropout(0.4)(x)
    outputs = Dense(1, activation='sigmoid')(x)

    return Model(inputs, outputs)

def compile_transfer_learning_model(model):
    """Compile with focal loss and class weights"""
    optimizer = optimizers.Adam(learning_rate=0.0001)
    model.compile(
        loss='binary_focal_crossentropy',  # Built-in focal loss
        optimizer=optimizer,
        metrics=['accuracy', 'recall', 'precision']
    )
    return model

# 2. Manual Class Balancing
def balance_classes(X_train, y_train):
    """Manual oversampling of minority class"""
    cancer_idx = np.where(y_train == 1)[0]
    non_cancer_idx = np.where(y_train == 0)[0]

    # Oversample cancer cases to match non-cancer count
    oversample_factor = len(non_cancer_idx) // len(cancer_idx)
    oversampled_cancer = np.repeat(cancer_idx, oversample_factor)

    balanced_idx = np.concatenate([non_cancer_idx, oversampled_cancer])
    np.random.shuffle(balanced_idx)

    return X_train[balanced_idx], y_train[balanced_idx]

# 3. Enhanced Training with Balanced Batches
from tensorflow.keras.utils import Sequence
import numpy as np

class CancerCentricGenerator(Sequence):
    """Keras-compatible generator with cancer-focused sampling"""
    def __init__(self, X, y, batch_size=32, augment=True):
        self.X = X
        self.y = y
        self.batch_size = batch_size
        self.augment = augment
        self.cancer_indices = np.where(y == 1)[0]
        self.non_cancer_indices = np.where(y == 0)[0]

        # Initialize augmentation
        self.augmentor = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.3,
            height_shift_range=0.3,
            shear_range=0.3,
            zoom_range=0.3,
            horizontal_flip=True,
            vertical_flip=True,
            brightness_range=[0.5, 1.5],
            fill_mode='nearest'
        )

    def __len__(self):
        return int(np.ceil(len(self.cancer_indices) / (self.batch_size//2)))

    def __getitem__(self, idx):
        # Get 50% cancer samples with augmentation
        cancer_batch_idx = np.random.choice(
            self.cancer_indices,
            size=self.batch_size//2,
            replace=True  # Allow oversampling
        )
        cancer_X = self.X[cancer_batch_idx]
        cancer_y = self.y[cancer_batch_idx]

        # Apply augmentation to cancer samples
        cancer_X_aug = np.stack([
            self.augmentor.random_transform(x) for x in cancer_X
        ])

        # Get 50% non-cancer samples without augmentation
        non_cancer_batch_idx = np.random.choice(
            self.non_cancer_indices,
            size=self.batch_size//2,
            replace=False
        )
        non_cancer_X = self.X[non_cancer_batch_idx]
        non_cancer_y = self.y[non_cancer_batch_idx]

        # Combine and shuffle
        X_batch = np.concatenate([cancer_X_aug, non_cancer_X])
        y_batch = np.concatenate([cancer_y, non_cancer_y])

        indices = np.arange(len(X_batch))
        np.random.shuffle(indices)

        return X_batch[indices], y_batch[indices]

    def on_epoch_end(self):
        # Shuffle indices after each epoch
        np.random.shuffle(self.cancer_indices)
        np.random.shuffle(self.non_cancer_indices)


def train_transfer_learning_model(X_train, y_train, X_val, y_val):
    # Balance classes through manual oversampling
    X_balanced, y_balanced = balance_classes(X_train, y_train)

    # Create model
    model = create_transfer_learning_model(X_balanced.shape[1:])
    model = compile_transfer_learning_model(model)

    # Callbacks
    callbacks = [
        EarlyStopping(monitor='val_recall', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_recall', factor=0.5, patience=2)
    ]

    # Calculate steps per epoch
    steps = int(np.ceil(len(y_balanced) / 32))

    # Train with Keras-compatible generator
    train_gen = CancerCentricGenerator(X_balanced, y_balanced)
    history = model.fit(
        train_gen,
        steps_per_epoch=steps,
        epochs=30,
        validation_data=(X_val, y_val),
        callbacks=callbacks,
        verbose=1
    )

    return model, history


# Usage example:
# model, history = train_transfer_learning_model(X_train, y_train, X_val, y_val)

In [18]:
model_v4, history_v4 = train_transfer_learning_model(X_train, y_train, X_val, y_val)

# Evaluate the final model
evaluate_final_model(model_v4, X_test, y_test)

/Users/mattisweber/.pyenv/versions/3.10.6/envs/lewagon-breast-cancer-project/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
118/172 ━━━━━━━━━━━━━━━━━━━━ 34s 637ms/step - accuracy: 0.8925 - loss: 3.9755 - precision: 0.8991 - recall: 0.8826

/Users/mattisweber/.pyenv/versions/3.10.6/envs/lewagon-breast-cancer-project/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


172/172 ━━━━━━━━━━━━━━━━━━━━ 104s 526ms/step - accuracy: 0.9113 - loss: 3.8503 - precision: 0.9167 - recall: 0.9035 - val_accuracy: 0.6399 - val_loss: 4.0240 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 2/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 88s 504ms/step - accuracy: 0.9910 - loss: 2.4441 - precision: 0.9917 - recall: 0.9904 - val_accuracy: 0.6390 - val_loss: 3.0539 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 3/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 82s 478ms/step - accuracy: 0.9941 - loss: 1.4696 - precision: 0.9932 - recall: 0.9951 - val_accuracy: 0.6399 - val_loss: 2.5301 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 1.0000e-04
Epoch 4/30
172/172 ━━━━━━━━━━━━━━━━━━━━ 81s 469ms/step - accuracy: 0.9978 - loss: 0.9185 - precision: 0.9988 - recall: 0.9967 - val_accuracy: 0.6390 - val_loss: 2.2102 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 5.0000e-05
Epoch 5/30
172

### Simple Balanced Approach

In [ ]:
def train_medical_classifier(X_train, y_train, X_val, y_val):
    """Medical-focused training with proven techniques"""

    # Step 1: Calculate proper class weights (not too extreme)
    from sklearn.utils.class_weight import compute_class_weight

    class_weights = compute_class_weight(
        'balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    # Moderate the extreme weights
    class_weight_dict = {
        0: min(class_weights[0], 2.0),  # Cap at 2x
        1: min(class_weights[1], 10.0)  # Cap at 10x for cancer
    }

    # Step 2: Simple oversampling (not custom generator)
    cancer_indices = np.where(y_train == 1)[0]
    # Repeat cancer cases 3x (not extreme oversampling)
    repeat_indices = np.tile(cancer_indices, 3)
    all_indices = np.concatenate([np.arange(len(y_train)), repeat_indices])

    X_train_balanced = X_train[all_indices]
    y_train_balanced = y_train[all_indices]

    # Step 3: Create model with proven architecture
    model = create_transfer_learning_model(X_train.shape[1:])

    # Step 4: Use standard binary crossentropy (focal loss was too aggressive)
    optimizer = optimizers.Adam(learning_rate=0.0001)
    model.compile(
        loss='binary_crossentropy',  # Back to standard loss
        optimizer=optimizer,
        metrics=['accuracy', 'recall', 'precision']
    )

    # Step 5: Standard augmentation
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.1,
        fill_mode='nearest'
    )

    # Step 6: Train with class weights only
    history = model.fit(
        datagen.flow(X_train_balanced, y_train_balanced, batch_size=32),
        steps_per_epoch=len(X_train_balanced) // 32,
        epochs=20,  # Shorter training
        class_weight=class_weight_dict,
        validation_data=(X_val, y_val),
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
        ],
        verbose=1
    )

    return model, history

model_v5, history_v5 = train_medical_classifier(X_train, y_train, X_val, y_val)
# Evaluate the final model
evaluate_final_model(model_v5, X_test, y_test)

/Users/mattisweber/.pyenv/versions/3.10.6/envs/lewagon-breast-cancer-project/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
347/347 ━━━━━━━━━━━━━━━━━━━━ 406s 1s/step - accuracy: 0.3956 - loss: nan - precision: 0.6959 - recall: 0.2010 - val_accuracy: 0.6399 - val_loss: nan - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/20
  1/347 ━━━━━━━━━━━━━━━━━━━━ 9:18 2s/step - accuracy: 0.3438 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00

/Users/mattisweber/.pyenv/versions/3.10.6/envs/lewagon-breast-cancer-project/lib/python3.10/site-packages/keras/src/trainers/epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


347/347 ━━━━━━━━━━━━━━━━━━━━ 14s 36ms/step - accuracy: 0.3438 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.6399 - val_loss: nan - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/20
347/347 ━━━━━━━━━━━━━━━━━━━━ 387s 1s/step - accuracy: 0.3239 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.6399 - val_loss: nan - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 4/20
347/347 ━━━━━━━━━━━━━━━━━━━━ 16s 41ms/step - accuracy: 0.3125 - loss: nan - precision: 0.0000e+00 - recall: 0.0000e+00 - val_accuracy: 0.6399 - val_loss: nan - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
37/37 ━━━━━━━━━━━━━━━━━━━━ 20s 364ms/step

Final Test Set Evaluation:
                            precision    recall  f1-score   support

Non-Cancer (Benign+Normal)       0.64      1.00      0.78       760
                    Cancer       0.00      0.00      0.00       424

                  accuracy                           0.64      1184
  

/Users/mattisweber/.pyenv/versions/3.10.6/envs/lewagon-breast-cancer-project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mattisweber/.pyenv/versions/3.10.6/envs/lewagon-breast-cancer-project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/mattisweber/.pyenv/versions/3.10.6/envs/lewagon-breast-cancer-project/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with